In [10]:
import cv2
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io
import heapq
import xlsxwriter
from xlutils.copy import copy    
from xlrd import open_workbook
import os
import math
from scipy import ndimage

In [11]:
# Search all data with _highBW.mat and save to files
path = os.path.abspath("/media/peppermint/easystore/123/Patients")
print(path)
files = []
paths=[];
# Find file with _ute.ni 

for r, d, f in os.walk(path):
    d.sort()
    for file in f:
        if '.mat' in file:
            index=-3;
            name=[];
            find_=False;
            #print(r)
            while(not find_):
                if(file[index]=='.'):
                    find_= True;
                index=index-1;
                if(find_):
                    name.append(file[0:index+1]);
            files.append(name)
            paths.append(r)


/media/peppermint/easystore/123/Patients


In [16]:
excel_ro = open_workbook("/home/peppermint/Desktop/Full_scan.xls")
excel = copy(excel_ro)  # creates a writeable copy
# find the last line
sheet1 = excel.get_sheet(0)  # get a first sheet
tempt_sheet = excel_ro.sheet_by_index(0)
colx = 0
rowx = tempt_sheet.nrows;



for num_file in range(len(files)):
#for num_file in range(10):
    # load data
    try:   
        current_path = paths[num_file]+'/'+str(files[num_file][0])+'.mat';
        #print(current_path)
        mat = scipy.io.loadmat(current_path)
        bar2gas = mat['bar2gas']
        mask_reg = mat['mask_reg']


        total_mean=[];
        for ij in range(128):

            bar2gas_current = ndimage.rotate(bar2gas[ij,:,:],90);
            mask_current = ndimage.rotate(mask_reg[ij,:,:],90);

            if(sum(sum(mask_current))>=900):
                current_img=bar2gas_current ;

                output=cv2.connectedComponentsWithStats(mask_current,4)

                num_labels = output[0];
                labels_im = output[1];
                stats=output[2];
                centroid=output[3];

                if(num_labels<=2):
                    continue

                area=stats[:,4];
                # Delete the background label.
                area=area[1:]

                #Choose the label with largest and second largest except backgrounf
                index_label=np.array(heapq.nlargest(2, range(len(area)), key=area.__getitem__))+1;

                # Find which is left or right
                index_1 = index_label[0]
                index_2 = index_label[1]
                if (centroid[index_1,0]<centroid[index_2,0]):
                    left_label=index_1;
                    right_label=index_2;
                else:
                    left_label=index_2;
                    right_label=index_1;

                # Instant of Do morphology we create distance image 

                dist_transform = cv2.distanceTransform(mask_current,cv2.DIST_L2,3)
                [m,n] = current_img.shape
                sum_left_out=0;
                sum_right_out=0;
                sum_left_in=0;
                sum_right_in=0;

                num_left_out=0;
                num_right_out=0;
                num_left_in=0;
                num_right_in=0;

                for i in range(m):
                    for j in range(n):

                        if(labels_im[i,j]==left_label ):

                            if((dist_transform[i,j]>0 and dist_transform[i,j]<=3)):
                                sum_left_out = sum_left_out+ current_img[i,j];
                                num_left_out= num_left_out+1;
                            else: 
                                sum_left_in = sum_left_in+ current_img[i,j];
                                num_left_in= num_left_in+1;


                        elif (labels_im[i,j]==right_label):

                            if((dist_transform[i,j]>0 and dist_transform[i,j]<=3)):
                                sum_right_out = sum_right_out+ current_img[i,j];
                                num_right_out= num_right_out+1;
                            else:
                                sum_right_in = sum_right_in+ current_img[i,j];
                                num_right_in= num_right_in+1;


                mean=[];
                have_nan = False;

                if num_left_out!=0:
                    mean.append(sum_left_out/num_left_out);
                else:
                    mean.append(np.nan);
                    have_nan = True;

                if num_left_in!=0:
                    mean.append(sum_left_in/num_left_in);
                else:
                    mean.append(np.nan);
                    have_nan = True;

                if num_right_out!=0:
                    mean.append(sum_right_out/num_right_out);
                else:
                    mean.append(np.nan);
                    have_nan = True;    

                if num_right_in!=0:
                    mean.append(sum_right_in/num_right_in);
                else:
                    mean.append(np.nan);
                    have_nan = True;


                #if(not have_nan):
                #    total_mean.append(mean);
                total_mean.append(mean);

        total_mean=np.array(total_mean);
        total_mean=np.nanmean(total_mean,axis=0)
        #total_mean=np.mean(total_mean,axis=0)

        print(total_mean)
        name = files[num_file][0];
        colx = 0
        try:
            sheet1.write(rowx,colx, name)
            sheet1.write(rowx,colx+1, total_mean[0])
            sheet1.write(rowx,colx+2, total_mean[1])
            sheet1.write(rowx,colx+3, total_mean[2])
            sheet1.write(rowx,colx+4, total_mean[3])
            rowx = rowx+1;
        except:
            print('Something not right')
    except:
        print('Something not right')

excel.save("/home/peppermint/Desktop/Full_scan.xls")


[0.44843124 0.48651465 0.59772321 0.64035961]
[0.49034144 0.48482762 0.58537973 0.61098397]
[0.61570578 0.66219422 0.71046381 0.77967141]
[0.54511071 0.60269788 0.80925437 0.86894214]
[0.38803347 0.4118871  0.459798   0.5135132 ]
[0.53399953 0.5729237  0.64976843 0.67374766]
[0.66934607 0.62849702 0.65596723 0.64033829]
[0. 0. 0. 0.]
[1.16003811 1.26023361 1.31375625 1.33069169]
[1.29006337 1.39922766 1.30743948 1.41153393]
[0.62609169 0.66143093 0.63305358 0.67963262]
[0.68458839 0.75216802 0.71445337 0.7822859 ]
[0.30271268 0.44408645 0.2749762  0.41993407]
[0.27387331 0.49958119 0.3498613  0.52494759]
[0.12545753 0.19970707 0.13415518 0.18627708]
[0.1644806  0.20455302 0.17028162 0.18111429]
[0.09831396 0.14530689 0.09739562 0.17770246]
[0.16894038 0.23039399 0.16528383 0.21721317]
[0.15753475 0.23392608 0.12191709 0.17834256]
[0.26435476 0.26533165 0.22664124 0.24742646]
[0.50579486 0.52343451 0.51298108 0.52309432]
[0.39196065 0.67469131 0.35186457 0.54147226]
[0.79516357 0.865256

[1.35792685 1.39830882 1.26051443 1.29127592]
[1.11570559 1.34199732 1.49658856 1.61258551]
[1.07520782 1.17172952 1.31560841 1.37377762]
[0.92245958 0.95917057 1.09330589 1.12416609]
[0.91815162 0.94599092 1.09056376 1.09386181]
[0.73547045 0.82703861 0.79991439 0.86408705]
[0.57706722 0.69512355 0.59023184 0.65208561]
[0.60745041 0.66132552 0.74085251 0.83561783]
[0.59358    0.68302316 0.80372123 0.87834858]
[0.49686389 0.55718903 0.49474785 0.55691515]
[0.58497822 0.60922823 0.50328359 0.56184882]
[0.79262911 0.78696868 0.67805639 0.77860578]
[0.67074171 0.74564039 0.52230457 0.63237357]
[0.91867175 0.9012522  0.78257198 0.81822673]
[0.87477516 0.89994816 0.75982256 0.90417233]
[0.6862024  0.70184091 0.54463406 0.6452428 ]
[0.6862024  0.70184091 0.54463406 0.6452428 ]
[0.68476263 0.81027429 0.6351876  0.73574346]
[0.7149221  0.89394191 0.61528207 0.81790597]
[0.85050434 0.92385288 0.77451628 0.82831614]
[0.53013992 0.5595222  0.55283185 0.60221378]
[0.5921895  0.63368686 0.66553664 

In [30]:
excel_ro = open_workbook("/home/peppermint/Desktop/Full_scan3.xls")
excel = copy(excel_ro)  # creates a writeable copy
# find the last line
sheet1 = excel.get_sheet(0)  # get a first sheet
tempt_sheet = excel_ro.sheet_by_index(0)
colx = 0
rowx = tempt_sheet.nrows;

# number of split 
ns =3;


for num_file in range(len(files)):
#for num_file in range(1):
# load data
    try:   
        current_path = paths[num_file]+'/'+str(files[num_file][0])+'.mat';
        #print(current_path)
        mat = scipy.io.loadmat(current_path)
        bar2gas = mat['bar2gas']
        mask_reg = mat['mask_reg']


        total_mean=[];
        for ij in range(128):

            bar2gas_current = bar2gas[:,:,ij];
            mask_current = mask_reg[:,:,ij];

            if(sum(sum(mask_current))>=900):
                current_img=bar2gas_current ;

                output=cv2.connectedComponentsWithStats(mask_current,4)

                num_labels = output[0];
                labels_im = output[1];
                stats=output[2];
                centroid=output[3];

                if(num_labels<=2):
                    continue

                area=stats[:,4];
                # Delete the background label.
                area=area[1:]

                #Choose the label with largest and second largest except backgrounf
                index_label=np.array(heapq.nlargest(2, range(len(area)), key=area.__getitem__))+1;

                # Find which is left or right
                index_1 = index_label[0]
                index_2 = index_label[1]
                if (centroid[index_1,0]<centroid[index_2,0]):
                    left_label=index_1;
                    right_label=index_2;
                else:
                    left_label=index_2;
                    right_label=index_1;

                top_left= stats[left_label,1];
                height_left = stats[left_label,3];

                top_right= stats[right_label,1];
                height_right = stats[right_label,3];

                [m,n] = current_img.shape

                sum_left=[0] * ns;
                sum_right=[0] * ns;

                num_left=[0] * ns;
                num_right=[0] * ns;

                for i in range(m):
                    for j in range(n):

                        if(labels_im[i,j]==left_label ):

                            for nlf in range(ns):
                                if(i >= top_left +(height_left/3)*(nlf) and i <= top_left+ (height_left/3)*(nlf+1)):
                                    sum_left[nlf] = sum_left[nlf]+ current_img[i,j];
                                    num_left[nlf]= num_left[nlf]+1;


                        elif (labels_im[i,j]==right_label):
                            for nlf in range(ns):
                                if(i >= top_right +(height_right/3)*(nlf) and i <= top_right+ (height_right/3)*(nlf+1)):
                                    sum_right[nlf] = sum_right[nlf]+ current_img[i,j];
                                    num_right[nlf]= num_right[nlf]+1;


                mean=[];
                have_nan = False;

                for nlf in range(ns):
                    if(num_left[nlf]!=0):
                        mean.append(sum_left[nlf]/num_left[nlf])
                    else:
                        mean.append(np.nan);

                for nlf in range(ns):
                    if(num_right[nlf]!=0):
                        mean.append(sum_right[nlf]/num_right[nlf])
                    else:
                        mean.append(np.nan);


                #if(not have_nan):
                #    total_mean.append(mean);
                total_mean.append(mean);

        total_mean=np.array(total_mean);
        total_mean=np.nanmean(total_mean,axis=0)
        #total_mean=np.mean(total_mean,axis=0)
        print(total_mean)
        name = files[num_file][0];
        colx = 0
        try:
            sheet1.write(rowx,colx, name)
            for nlf in range(ns*2): 
                sheet1.write(rowx,colx+nlf+1, total_mean[nlf])

            rowx = rowx+1;
        except:
            print('Something not right')
    except:
        print('Something not right')

excel.save("/home/peppermint/Desktop/Full_scan3.xls")


[0.52730229 0.49499516 0.462896   0.58987524 0.57198061 0.62038752]
[0.51514327 0.49966577 0.44156603 0.58096284 0.58974875 0.6012695 ]
[0.65117844 0.65826546 0.58382503 0.7233868  0.73301099 0.6687189 ]
[0.65301713 0.6310186  0.61383403 0.80926083 0.79072054 0.81245041]
[0.44588158 0.40120094 0.42297262 0.56780721 0.46012755 0.47650531]
[0.53854298 0.51098282 0.56645278 0.5983913  0.60470829 0.69094613]
[0.75095638 0.59014719 0.60053638 0.71516039 0.67409901 0.73190329]
[0. 0. 0. 0. 0. 0.]
[1.08482935 1.3376538  1.13762905 1.20346542 1.44338665 1.36790317]
[1.22085427 1.45698569 1.10529791 1.2927964  1.55491471 1.30323889]
[0.60060686 0.66265781 0.64127224 0.54916679 0.65083065 0.65965361]
[0.65643738 0.75112724 0.70020677 0.62445155 0.73553658 0.76836637]
[0.3852226  0.39117183 0.30295299 0.39493161 0.38008833 0.278643  ]
[0.37500421 0.40126749 0.38163436 0.4512404  0.45717687 0.41310085]
[0.26541027 0.23880582 0.13329763 0.20468542 0.24580328 0.14896701]
[0.24163486 0.24278436 0.153

[0.80479319 0.79230215 0.73141875 0.84097276 0.80963306 0.67783462]
[0.51704753 0.54409679 0.59940226 0.46760635 0.54038986 0.58565888]
[0.49920631 0.51956416 0.61740671 0.39754082 0.44128725 0.49832025]
[0.51704753 0.54409679 0.59940226 0.46760635 0.54038986 0.58565888]
[0.49920631 0.51956416 0.61740671 0.39754082 0.44128725 0.49832025]
[0.30310848 0.57080301 0.54748189 0.40051464 0.55176212 0.55608659]
[0.80158526 0.75670316 0.69275843 0.76051989 0.6622253  0.45982508]
[0.74899213 0.75512699 0.56582322 0.71267171 0.62758214 0.43924653]
[0.92470022 0.93750012 0.7911276  0.93850854 0.81411822 0.53127117]
[1.00151974 0.98750665 0.87465045 1.09980901 0.98282883 0.67492648]
[0.93568919 0.90864155 0.80400092 0.92142184 0.90215576 0.8033439 ]
[0.86961716 0.84437445 0.75840156 0.85978526 0.84472113 0.72190038]
[0.94263963 0.87237091 0.75251086 0.86125325 0.81146317 0.73138376]
[1.02068266 0.95040817 0.82306487 0.95303762 0.87842019 0.77842578]
[0.99926088 0.90774832 0.79746468 0.85944599 0.8

[0.47334491 0.51470553 0.51499749 0.50325815 0.50223371 0.59158638]
[0.54673424 0.62072734 0.59571338 0.53830988 0.598613   0.47202618]
[0.09356967 0.10884482 0.02676017 0.18156409 0.12599047 0.04345064]
[0.81122514 0.87084009 0.85235367 0.59721323 0.84118143 0.85130674]
[0.55264925 0.6204255  0.50141816 0.54688507 0.58117768 0.5109848 ]
[0.52697696 0.58358922 0.50821656 0.55947103 0.60596811 0.49341209]
[0.82944108 0.73950023 0.75694687 0.79292107 0.78420498 0.75003709]
[0.22506133 0.44348995 0.4186223  0.21643186 0.42600916 0.43677485]
[0.15728243 0.11538671 0.0767888  0.184135   0.10305276 0.13270115]
[0.54461252 0.62298102 0.46936685 0.53994177 0.63004255 0.42107016]
[0.61529641 0.63986987 0.60787301 0.66412618 0.70806705 0.68475668]
[0.5741285  0.46333526 0.34153003 0.68186728 0.51989191 0.35108144]
[0.64907377 0.69565094 0.71384992 0.61523939 0.70253498 0.75562507]
[1.00207261 1.00004173 1.04217099 1.02952396 1.19282273 1.11206205]
[0.93737309 0.95329048 0.9171557  0.93711031 0.9